# HMC in ND with LF-BFGS autotuning of the mass matrix

This notebook implements an autotuning HMC for an N-dimensional distribution based on limited-memory, factorised BFGS (LF-BFGS) updating of the inverse Hessian.

Compared to the BFGS and F-BFGS variants of autotuning, the method implemented here is considerably more complex for two reasons: (1) The initial matrix factor $\mathbf{S}_0$ can be updated. While this help to effectively produce longer-term memory, it also can lead to a poorly conditioned Hessian, and therefore to leap-frog instability. (2) The leap-frog integration time step is adaptive and depends on the average acceptance rate. Also this may lead to numerical instability if not done conservatively.

It is therefore advisable to not have the updating of $\mathbf{S}_0$ and the adaptive time stepping working at the same time. One possible strategy is to first do a few updates of $\mathbf{S}_0$ during the first 'few' samples, and then start with the adaptive time stepping.

## 0. Import packages

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import testfunctions
import samplestatistics

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 50})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

## 1. Input

We first define several input parameters, including the model space dimension, the initial inverse mass matrix $\mathbf{M}^{-1}$, the total number of samples, the number of leapfrog timesteps, and the length of the timestep.

In [ ]:
import input_parameters
reload(input_parameters)
test_function,dim,N,Nit,dt0,m0,Minv,autotune,ell,update_interval,preco,S0_min,plot_interval,dimension1,dimension2,m1_min,m1_max,m2_min,m2_max=input_parameters.input_parameters()

## 2. Class for LF-BFGS autotuning

This class takes care of the limited-memory, factorised BFGS updating. The class must be initialised with the first model and the first gradient. The *update* function then takes the next model and gradient, and computes updates of the approximate matrix factor $\mathbf{S}$, of the approximate Hessian $\mathbf{H}$, and of the approximate inverse Hessian $\mathbf{H}^{-1}$.

**Call to caution**: There is an experimental component in this class. In principle, BFGS updates can only be made when $\mathbf{s}_k^T\mathbf{y}>0$. However, when this quantity is very small, the resulting Hessian approximation may still be close to singular. Empirically, it is better for stability to choose $\mathbf{s}_k^T\mathbf{y}>\gamma$, with some tuning parameter $\gamma>0$. For many examples, $\gamma=2$ works very well. In general, it seems to make sense to check with a little test run, without autotuning, what typical values of $\mathbf{s}_k^T\mathbf{y}$ actually are.

In [ ]:
class lfbfgs:
    
    # Initialisation. ==================================================================
    
    def __init__(self,dim,Minv,k):
        """
        Initialise the LF-BFGS iteration.
        
        :param dim: number of model-space dimensions
        :param Minv: diagonal elements of the initial inverse mass matrix
        :param k: maximum number of vectors to be stored
        """
        
        self.dim=dim # Model space dimension.
        self.k=k # Maximum number of vectors.
        self.i=0 # Currently stored vectors.
        
        self.s=np.zeros((dim,k)) # s vectors.
        self.y=np.zeros((dim,k)) # y vectors.
        self.u=np.zeros((dim,k)) # u vectors.
        self.v=np.zeros((dim,k)) # v vectors.
        self.vTu=np.zeros(k) # Precomputed 1+vT*u.
        
        self.S0=np.ones(dim) # Diagonal components of the initial S factor.
        for i in range(dim):
            self.S0[i]=1.0/np.sqrt(Minv[i])
            
    # Compute and store new vectors u and v. ===========================================
        
    def put_sy(self,s,y):
        """
        Put in a new pair of vectors s (model difference) and y (gradient difference).
        
        :param s: current model difference vector
        :param y: current gradient difference vector
        """
        
        #===============================================================================
        # Compute new s, y, u, v vectors.
        #===============================================================================
        
        sy=np.dot(s,y)
        #print(sy)
        
        # Do nothing unless rho is positive.
        if sy>10.0 and self.k>0:
             
            # Precompute inverse Hessian-vector product.
            Hinv_y=self.Hinv(y)
        
            # Auxiliary scalars and vectors.
            rho=1.0/sy
            gamma2=rho**2 * np.dot(y,Hinv_y) + rho
            beta=gamma2 * np.dot(s,self.H(s))
            
            # Check if certain quantities are actually positive.
            if (gamma2>0.0) and (beta>0.0):
            
                #theta=np.sqrt(rho/(beta*gamma2)) # Here, one may also choose the positive square root. Empirically, the negative ones works much better because it leads to a diagonally dominant S, which is better for preconditioning.
                theta=-np.sqrt(rho/(beta*gamma2)) 
                a=np.sqrt(gamma2)*s
                b=(rho/np.sqrt(gamma2))*Hinv_y
            
                # Compute the current u and v.
                u=a
                v=-self.H(b+theta*a)
         
                #===============================================================================
                # Update s, y, u, v in the memory.
                #===============================================================================
        
                # When less then k vector pairs are stored, we increase the index of stored pairs and add the new pair.
                if self.i<self.k:
                    self.i+=1
                # Otherwise we kick out the pair with the lowest index by rolling to the left and over-writing the last pair.
                else:
                    self.s=np.roll(self.s, -1, axis=1)
                    self.y=np.roll(self.y, -1, axis=1)
                    self.u=np.roll(self.u, -1, axis=1)
                    self.v=np.roll(self.v, -1, axis=1)
                    self.vTu=np.roll(self.vTu,-1)

                self.s[:,self.i-1]=s
                self.y[:,self.i-1]=y
                self.u[:,self.i-1]=u
                self.v[:,self.i-1]=v
                self.vTu[self.i-1]=1.0+np.dot(v,u)
                
            else:
                print('LF-BFGS check failed (gamma=%f, beta=%f)' % (gamma2,beta))
            
        else: 
            print('LF-BFGS check failed (1/rho=%f)' % sy)

    # Implement matrix-vector products. ===============================================
    
    def S(self,h):
        """
        S*h, product of factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i):
            h=h-self.v[:,i]*np.dot(self.u[:,i],h)/self.vTu[i]
        return h*self.S0
    
    def Sn(self,h,n):
        """
        S*h, product of factor S with some vector h using a smaller number n of vectors
        :param h: vector of dimension self.dim
        :param n: number of vectors to take into account
        """
        
        for i in range(n):
            h=h-self.v[:,i]*np.dot(self.u[:,i],h)/self.vTu[i]
        return h*self.S0
    
    def ST(self,h):
        """
        S^T*h, product of transposed factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i-1,-1,-1):
            h=h-self.u[:,i]*np.dot(self.v[:,i],h)/self.vTu[i]
        return h*self.S0
        
    def Sinv(self,h):
        """
        S^-1*h, product of inverse factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i-1,-1,-1):
            h=h+self.v[:,i]*np.dot(self.u[:,i],h)
        return h/self.S0
    
    def SinvT(self,h):
        """
        S^-T*h, product of inverse transposed factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i):
            h=h+self.u[:,i]*np.dot(self.v[:,i],h)
        return h/self.S0
    
    def H(self,h):
        """
        H*h, product of Hessian approximation H with some vector h
        :param h: vector of dimension self.dim
        """
        
        h=self.ST(h)
        return self.S(h)
    
    def Hinv(self,h):
        """
        Hinv*h, product of inverse Hessian approximation H with some vector h
        :param h: vector of dimension self.dim
        """
        
        h=self.Sinv(h)
        return self.SinvT(h)
    
    # Implement log determinant of S. ==================================================
    def logdet(self):
        """
        Current log of the determinant of S
        """
        
        logdet=0.0
        for i in range(self.i):
            alpha=1.0/(1.0+np.dot(self.u[:,i],self.v[:,i]))
            logdet+=np.log(alpha**2)
            
        return logdet

## 3. Function for updating the initial matrix factor $\mathbf{S}_0$

The updating of the initial matrix factor $\mathbf{S}_0$, which may be done repeatedly, can on one hand greatly accelerate convergence. On the other hand, it can be a major source of instability because the resulting Hessian approximations may be poorly conditioned. This updating mechanism distinguishes the limited-memory variant of the autotuning algorithm from its matrix-based variants using (F-)BFGS.

The updating implemented here only updates the diagonal of $\mathbf{S}_0$ using the diagonal of the current Hessian approximation. Empirically it improves convergence when the diagonal is smoothed quite substantially.

In [ ]:
def update_S0(M,S0_min):
    """
    Iterative updating of the initial matrix factor S0.
    
    :param M: an lfbfgs object which needs to be updated.
    :param S0_min: minimum allowable diagonal entry of S0. Used for stabilisation.
    """
    
    # Get diagonal elements of the current inverse Hessian.
    Minv_new=np.zeros(M.dim)
    for i in range(M.dim):
        ei=np.zeros(M.dim)
        ei[i]=1.0
        Hinv_ii=M.Hinv(ei)[i]
        # Compute new initial diagonal elements from current ones, in case they are numbers.
        if np.isnan(Hinv_ii): Minv_new[i]=1.0/(M.S0[i]**2.0)
        else: Minv_new[i]=np.min(( Hinv_ii , 1.0/(S0_min**2.0) ))
        
    # Smooth diagonal. Empirically improves convergence.
    for k in range(100):
        Minv_new[0]=(2.0*Minv_new[0]+Minv_new[1])/3.0
        Minv_new[-1]=(2.0*Minv_new[-1]+Minv_new[-2])/3.0
        Minv_new[1:M.dim-1]=(Minv_new[0:M.dim-2]+Minv_new[1:M.dim-1]+Minv_new[2:M.dim])/3.0
       
    # Initialise new lfbfgs class member.
    M_new=lfbfgs(M.dim,Minv_new,M.k)
    
    #plt.plot(Minv_new)
    #plt.show()
    
    # Return the new lfbfgs class member.
    return M_new

## 4. Leapfrog integrator

For clarity, we define the leap-frog integrator as a separate function. The numerical stability of leapfrog depends on the eigenvalues of the Hessian, and so it may change in the course of the sampling. The implementation below tries to catch potential instabilities by resetting the model and momentum to a random value in case the model values grow beyond resonable values. This may need adjustment in the code.

In [ ]:
def leapfrog(m,p,Nt,dt,lfbfgs,fct,plot=False):
    """
    Leapfrog time stepping with possible plotting of the trajectory.
    
    :param m: current model
    :param p: current momentum
    :param Nt: maximum number of time steps
    :param dt: time step lenth
    :param lfbfgs: LFBFGS object
    :param fct: test function object
    :param plot: plot trajectory if True
    """
    
    # Plot probability density in the background.
    if plot:
        fct.plotU(dim,dimension1,dimension2,m1_min,m1_max,m2_min,m2_max)
        plt.plot(m[dimension1],m[dimension2],'bo',MarkerSize=15)
    
    # Evaluate initial gradient.
    J=fct.J(m)
    
    # Determine randomised integration length.
    if Nt>=2: Nti=np.int(Nt*(1.0-0.5*np.random.rand()))
    else: Nti=Nt
    
    # Leapfrog integration.
    for k in range(Nti):
        
        # Save initial model for later plotting and to catch instabilities.
        m_old=m.copy()
        
        # Perform one time step.
        p=p-0.5*dt*J
        m=m+dt*lfbfgs.Hinv(p)
        J=fct.J(m)
        p=p-0.5*dt*J
        
        # Catch potential instability.
        if np.max(np.abs(m))>100.0:
            print('LEAPFROG INSTABILITY')
            print(np.max(np.abs(m)))
            m=np.random.randn(np.shape(m)[0])
            p=np.random.randn(np.shape(m)[0])
            return m, p
        
        # Plot trajectory segment.
        if plot: 
            if k==0: print('number of time steps: %d' % Nti)
            plt.plot([m_old[dimension1],m[dimension1]],[m_old[dimension2],m[dimension2]],'r',linewidth=3)
            plt.plot(m[dimension1],m[dimension2],'kx',markersize=15)
        
    return m, p

## 6. HMC initialisations

Before running the actual HMC sampler, we perform several initialisations. This includes the test function class, the first random model $\mathbf{m}$, and the corresponding gradient of the potential energy $\mathbf{g}=\nabla U$. With this, we can initialise the BFGS class, which takes $\mathbf{m}$ and $\mathbf{g}$ as input.

In [ ]:
# Initialisation. =============================================================

# Test function class.
fct=testfunctions.f(dim,test_function)

# Initial time step.
dt=dt0

# Number of accepted models.
accept=np.zeros(N)
# Current averaged acceptance rate.
average_accept=np.ones(N)
# Current time step.
time_step=dt*np.ones(N)

# Initial model (deterministic or randomly chosen).
m=m0.copy()

# Posterior statistics.
stats=samplestatistics.stats(dimension1,dimension2,N)
stats.get(m,0.0,0)

# Initialise LF-BFGS.
g=fct.J(m)
M=lfbfgs(dim,Minv.diagonal(),ell)

# Specific matrix elements for monitoring.
m11=Minv[dimension1,dimension1]*np.ones(N)
m22=Minv[dimension2,dimension2]*np.ones(N)

# Unit vectors needed to obtain specific mass matrix components for monitoring.
e1=np.zeros(dim)
e2=np.zeros(dim)
e1[dimension1]=1.0
e2[dimension2]=1.0

# Last iteration when the adaptive time step was changed.
it_change_dt=0

# Talk or not.
verbose=False

## 7. Run HMC

We finally run the HMC sampler. In each iteration, we first produce radom momenta $\mathbf{p}$ from a normal distribution with covariance chosen to be the BFGS-updated inverse mass matrix $\mathbf{M}^{-1}$, which is defined to be the inverse Hessian $\mathbf{H}^{-1}$ of the potential energy $U$. 

Using the mass matrix, we compute energies and run a leapfrog iteration to solve Hamilton's equations. Following this, we compute the energies of the proposed model and evaluate the modified Metropolis rule (in logarithimic form, to avoid over- or under-flow).

In [ ]:
start=time.time()

# Sampling. ===================================================================

for it in range(N-1):

    # Randomly choose momentum.
    p=np.random.randn(dim)
    p=M.S(p)
    
    # Evaluate energies.
    U=fct.U(m)
    K=0.5*np.dot(p,M.Hinv(p))
    H=U+K
    
    # Check if models and trajectories should be plotted.
    if (not it % plot_interval) and it>0: 
        plot=True
        print('iteration: %d' % it)
    else:
        plot=False
    
    # Run leapfrog iteration.
    m_new,p_new=leapfrog(m,p,Nit,dt,M,fct,plot)
    plt.show()
        
    # Plot proposed models.
    if plot:
        plt.subplots(1, figsize=(30,10))
        plt.plot(m_new,'k')
        plt.xlabel('model parameter index')
        plt.grid()
        plt.show()
        
    # Evaluate new energies.
    U_new=fct.U(m_new)
    K_new=0.5*np.dot(p_new,M.Hinv(p_new))
    H_new=U_new+K_new
    
    # Evaluate Metropolis rule in logarithmic form.
    alpha=np.minimum(0.0,H-H_new)
    if alpha>=np.log(np.random.rand(1)):
        # Accepted.
        if verbose: print('it=%d accepted' % it)
        accept[it]=1
        # Compute s and y vectors.
        s=m_new-m
        g_new=fct.J(m_new)
        y=g_new-g  
        # Update model and gradient.
        m=m_new
        g=g_new
        if autotune and (not it % update_interval): M.put_sy(s,y)
 
    # Preconditioning of the S0 diagonal.
    if autotune and preco and (not it % 5) and (it>=ell) and (it<=ell+30): M=update_S0(M,S0_min)
    #if autotune and preco and (it==ell): M=update_S0(M,S0_min)
        
    # Accumulate on-the-fly statistics.
    m11[it+1]=np.dot(e1,M.Hinv(e1))
    m22[it+1]=np.dot(e2,M.Hinv(e2))
    stats.get(m,M.logdet(),it+1)
    
    # Adaptive time stepping.
    Navg=20
    if autotune and (it>=Navg) and (it-it_change_dt>Navg/2):
        average_accept[it]=np.sum(accept[it-Navg+1:it+1])/np.float(Navg)
        time_step[it]=dt
        if verbose: print('it=%d average acceptance rate: %f' % (it,average_accept[it]))
        if average_accept[it]<0.35:
            dt=0.8*dt
            time_step[it]=dt
            print('--> time step reduced to %f in iteration %d' % (dt,it))
            it_change_dt=it
        elif average_accept[it]>0.95:
            dt=1.25*dt
            time_step[it]=dt
            print('--> time step increased to %f in iteration %d' % (dt,it))
            it_change_dt=it

# Store final diagonal of the mass matrix.
np.save('OUTPUT/S0.npy',M.S0)

# Output basic statistics.
stop=time.time()
print('acceptance rate: %f (%d of %d samples)' % (np.sum(accept)/np.float(N),np.sum(accept),N))
print('elapsed time: %f s' % (stop-start))

## 8. Analyse results

## 8.1. Acceptance and adaptive time stepping

In [ ]:
# Average acceptance rate.
plt.subplots(1, figsize=(20,10))
plt.plot(average_accept,'k')
plt.xlabel('sample index')
plt.ylabel('acceptance rate averaged over %d samples' % Navg)
plt.grid()
plt.savefig('OUTPUT/acceptance_rate.png', bbox_inches='tight', format='png')
plt.show()

# Adaptive time step.
plt.subplots(1, figsize=(20,10))
plt.plot(time_step,'k')
plt.xlabel('sample index')
plt.ylabel('time step')
plt.grid()
plt.savefig('OUTPUT/time_step.png', bbox_inches='tight', format='png')
plt.show()

# Acceptance per sample.
plt.subplots(1, figsize=(30,10))
plt.plot(accept,'kx')
plt.show()

### 8.2. Sample statistics collected on the fly

In [ ]:
stats.display()
stats.save()